In [80]:
import pandas as pd
import re
import ast
import numpy as np

from rapidfuzz import process, fuzz

In [81]:
wiki_movies = pd.read_csv("wikipedia_movie_franchises.csv", index_col = 0)
imdb_movies = pd.read_csv("imdb_movies_db.csv", index_col = 0)

In [82]:
wiki_movies = wiki_movies.dropna(axis=0, how='any')
wiki_movies.reset_index(inplace = True)
wiki_movies["release_year"] = wiki_movies["release_year"].apply(lambda x : int(x))

In [83]:
wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.strip())
wiki_movies["franchise_name"] = wiki_movies["franchise_name"].apply(lambda x : x.strip())

In [84]:
wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.replace("–", "-"))
wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.lower())

In [85]:
def lower_akas(row):
    if type(row) == str:
        row = row.lower()
        return row

In [86]:
imdb_movies["primaryTitle"] = imdb_movies["primaryTitle"].apply(lambda x : x.lower())
imdb_movies["originalTitle"] = imdb_movies["originalTitle"].apply(lambda x : x.lower())
imdb_movies["akas"] = imdb_movies["akas"].apply(lambda x : lower_akas(x))
imdb_movies["startYear"] = imdb_movies["startYear"].apply(lambda x : int(x) if x != "\\N" else np.nan)

In [87]:
wiki_filter = ["tconst", "titleType", "primaryTitle","originalTitle", "isAdult","startYear","endYear","runtimeMinutes","genres","averageRating","numVotes","akas","_merge", "all_names"]
imdb_filter = ["index", "franchise_name", "franchise_id", "movie_name", "release_year","_merge"]

In [88]:
imdb_copy = imdb_movies.copy()

In [89]:
#imdb_copy[imdb_copy["numVotes"] >100]

In [90]:
#imdb_copy

In [91]:
def filter_df(df, merge, cols):
    remainder = df[df["_merge"]== merge]
    remainder = remainder.drop(cols, axis = 1)
    return remainder

In [92]:
def extract_akas(row):
    # convert string in list format to list
    if type(row) == str:
        akas = ast.literal_eval(row)
        return akas

In [93]:
imdb_copy["akas"] = imdb_copy["akas"].apply(lambda x : extract_akas(x))

In [94]:
imdb_copy['all_names'] = np.empty((len(imdb_copy), 0)).tolist()

In [95]:
imdb_copy = imdb_copy.reset_index(drop=True)

In [96]:
for i in range(0, len(list(imdb_copy["primaryTitle"]))):
    imdb_copy['all_names'][i].append(imdb_copy['primaryTitle'][i])
    imdb_copy['all_names'][i].append(imdb_copy['originalTitle'][i])
    
    if type(imdb_copy['akas'][i]) == list:
        for x in imdb_copy['akas'][i]:
            imdb_copy['all_names'][i].append(x)
        
        
imdb_copy

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names
0,tt0000009,movie,miss jerry,miss jerry,0,1894.0,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]","[miss jerry, miss jerry, miss jerry, fräulein ..."
1,tt0000147,movie,the corbett-fitzsimmons fight,the corbett-fitzsimmons fight,0,1897.0,\N,100,"Documentary,News,Sport",5.3,469.0,"[the corbett-fitzsimmons fight, бой корбетта и...","[the corbett-fitzsimmons fight, the corbett-fi..."
2,tt0000502,movie,bohemios,bohemios,0,1905.0,\N,100,\N,4.1,15.0,[bohemios],"[bohemios, bohemios, bohemios]"
3,tt0000574,movie,the story of the kelly gang,the story of the kelly gang,0,1906.0,\N,70,"Action,Adventure,Biography",6.0,823.0,"[the story of the kelly gang, kelly bandájának...","[the story of the kelly gang, the story of the..."
4,tt0000591,movie,the prodigal son,l'enfant prodigue,0,1907.0,\N,90,Drama,4.4,20.0,"[l'enfant prodigue, the prodigal son]","[the prodigal son, l'enfant prodigue, l'enfant..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290234,tt9916270,movie,il talento del calabrone,il talento del calabrone,0,2020.0,\N,84,Thriller,5.8,1414.0,"[il talento del calabrone, the talent of the h...","[il talento del calabrone, il talento del cala..."
290235,tt9916362,movie,coven,akelarre,0,2020.0,\N,92,"Drama,History",6.4,5293.0,"[boszorkánygyülekezet, coven, les sorcières d'...","[coven, akelarre, boszorkánygyülekezet, coven,..."
290236,tt9916428,movie,the secret of china,hong xing zhao yao zhong guo,0,2019.0,\N,\N,"Adventure,History,War",3.8,14.0,"[the secret of china, hong xing zhao yao zhong...","[the secret of china, hong xing zhao yao zhong..."
290237,tt9916538,movie,kuambil lagi hatiku,kuambil lagi hatiku,0,2019.0,\N,123,Drama,8.6,7.0,[kuambil lagi hatiku],"[kuambil lagi hatiku, kuambil lagi hatiku, kua..."


In [97]:
imdb_copy = imdb_copy.explode("all_names")

In [98]:
all_merged = pd.merge(wiki_movies, imdb_copy, left_on=["movie_name", "release_year"], right_on=["all_names", "startYear"], how="outer", indicator = True)
wiki_remainder = filter_df(all_merged, "left_only", wiki_filter)
imdb_remainder = filter_df(all_merged, "right_only", imdb_filter)
all_merged[all_merged["_merge"]=="both"]

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
0,0.0,f0,The Aldrich Family,what a life,1939.0,tt0032123,movie,what a life,what a life,0.0,1939.0,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
1,0.0,f0,The Aldrich Family,what a life,1939.0,tt0032123,movie,what a life,what a life,0.0,1939.0,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
2,0.0,f0,The Aldrich Family,what a life,1939.0,tt0032123,movie,what a life,what a life,0.0,1939.0,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
3,1.0,f0,The Aldrich Family,life with henry,1940.0,tt0033834,movie,life with henry,life with henry,0.0,1940.0,\N,80,"Comedy,Family,Music",6.0,49.0,"[life with henry, henry quería ir a alaska, he...",life with henry,both
4,1.0,f0,The Aldrich Family,life with henry,1940.0,tt0033834,movie,life with henry,life with henry,0.0,1940.0,\N,80,"Comedy,Family,Music",6.0,49.0,"[life with henry, henry quería ir a alaska, he...",life with henry,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16309,8163.0,f1343,Super Sentai,super hero taisen gp: kamen rider 3,2015.0,tt4282466,movie,super hero taisen gp: kamen rider 3,super hero taisen gp: kamen rider 3,0.0,2015.0,\N,95,Action,6.5,85.0,"[super hero taisen gp: kamen rider 3, スーパーヒーロー...",super hero taisen gp: kamen rider 3,both
16322,8176.0,f1343,Super Sentai,kishiryu sentai ryusoulger special chapter: me...,2021.0,tt13681618,movie,kishiryu sentai ryusoulger special chapter: me...,kishiryuu sentai ryuusoujâ tokubetsuhen: memor...,0.0,2021.0,\N,15,"Action,Fantasy",6.4,6.0,[kishiryuu sentai ryuusoujâ tokubetsuhen: memo...,kishiryu sentai ryusoulger special chapter: me...,both
16323,8176.0,f1343,Super Sentai,kishiryu sentai ryusoulger special chapter: me...,2021.0,tt13681618,movie,kishiryu sentai ryusoulger special chapter: me...,kishiryuu sentai ryuusoujâ tokubetsuhen: memor...,0.0,2021.0,\N,15,"Action,Fantasy",6.4,6.0,[kishiryuu sentai ryuusoujâ tokubetsuhen: memo...,kishiryu sentai ryusoulger special chapter: me...,both
16326,8179.0,f1343,Super Sentai,kaizoku sentai: ten gokaiger,2021.0,tt14879560,movie,kaizoku sentai: ten gokaiger,kaizoku sentai ten gôkaijâ,0.0,2021.0,\N,61,"Action,Adventure,Comedy",8.6,73.0,"[kaizoku sentai ten gokaiger, kaizoku sentai t...",kaizoku sentai: ten gokaiger,both


In [99]:
all_merged_both = all_merged[all_merged["_merge"]=="both"]
all_merged_both = all_merged_both.drop_duplicates(subset = ["index", "tconst"],keep = "first")
all_merged_both

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
0,0.0,f0,The Aldrich Family,what a life,1939.0,tt0032123,movie,what a life,what a life,0.0,1939.0,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
3,1.0,f0,The Aldrich Family,life with henry,1940.0,tt0033834,movie,life with henry,life with henry,0.0,1940.0,\N,80,"Comedy,Family,Music",6.0,49.0,"[life with henry, henry quería ir a alaska, he...",life with henry,both
6,2.0,f0,The Aldrich Family,henry aldrich for president,1941.0,tt0033708,movie,henry aldrich for president,henry aldrich for president,0.0,1941.0,\N,75,"Comedy,Family",6.6,146.0,"[henry aldrich para presidente, henry aldrich ...",henry aldrich for president,both
9,3.0,f0,The Aldrich Family,"henry aldrich, editor",1942.0,tt0034842,movie,"henry aldrich, editor","henry aldrich, editor",0.0,1942.0,\N,72,"Comedy,Drama,Family",6.4,150.0,"[henry periodista, henry aldrich, editor]","henry aldrich, editor",both
12,4.0,f0,The Aldrich Family,henry and dizzy,1942.0,tt0034844,movie,henry and dizzy,henry and dizzy,0.0,1942.0,\N,71,"Comedy,Family",7.2,58.0,[henry and dizzy],henry and dizzy,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16299,8157.0,f1343,Super Sentai,kamen rider × super sentai × space sheriff: su...,2013.0,tt2632184,movie,kamen rider × super sentai × space sheriff: su...,kamen raidâ × sûpâ sentai × uchû keiji: supâ h...,0.0,2013.0,\N,92,"Action,Adventure,Comedy",6.3,129.0,[kamen raidâ × sûpâ sentai × uchû keiji: supâ ...,kamen rider × super sentai × space sheriff: su...,both
16305,8162.0,f1343,Super Sentai,ressha sentai toqger vs. kyoryuger: the movie,2015.0,tt4152148,movie,ressha sentai toqger vs. kyoryuger: the movie,ressha sentai tokkyûjâ tai kyôryûjâ za mûbî,0.0,2015.0,\N,64,Action,7.5,32.0,"[ressha sentai tokkyûjâ tai kyôryûjâ za mûbî, ...",ressha sentai toqger vs. kyoryuger: the movie,both
16307,8163.0,f1343,Super Sentai,super hero taisen gp: kamen rider 3,2015.0,tt4282466,movie,super hero taisen gp: kamen rider 3,super hero taisen gp: kamen rider 3,0.0,2015.0,\N,95,Action,6.5,85.0,"[super hero taisen gp: kamen rider 3, スーパーヒーロー...",super hero taisen gp: kamen rider 3,both
16322,8176.0,f1343,Super Sentai,kishiryu sentai ryusoulger special chapter: me...,2021.0,tt13681618,movie,kishiryu sentai ryusoulger special chapter: me...,kishiryuu sentai ryuusoujâ tokubetsuhen: memor...,0.0,2021.0,\N,15,"Action,Fantasy",6.4,6.0,[kishiryuu sentai ryuusoujâ tokubetsuhen: memo...,kishiryu sentai ryusoulger special chapter: me...,both


In [100]:
imdb_remainder = imdb_remainder[~imdb_remainder["tconst"].isin(all_merged_both["tconst"])]

In [101]:
no_year = pd.merge(wiki_remainder, imdb_remainder, left_on=["movie_name"], right_on=["all_names"], how="outer", indicator = True)
wiki_remainder = filter_df(all_merged, "left_only", wiki_filter)
imdb_remainder = filter_df(all_merged, "right_only", imdb_filter)
no_year

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
0,11.0,f1,Coffin Joe,at midnight i'll take your soul,1963.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,15.0,f1,Coffin Joe,the end of man,1970.0,tt0067099,movie,finis hominis,finis hominis,0.0,1971.0,\N,79,"Comedy,Drama,Mystery",5.7,373.0,"[end of man, the end of man, finis hominis, en...",the end of man,both
2,26.0,f2,The Crime Club,the last express,1938.0,tt0037776,movie,the hidden eye,the hidden eye,0.0,1945.0,\N,69,"Action,Crime,Mystery",6.2,397.0,"[the hidden eye, perfume do oriente, l'oeil ca...",the last express,both
3,43.0,f3,Fast & Furious,fast x,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,48.0,f4,Gingerdead Man vs. Evil Bong,evil bong 3d: the wrath of bong,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842399,NaN,NaN,NaN,NaN,NaN,tt9916538,movie,kuambil lagi hatiku,kuambil lagi hatiku,0.0,2019.0,\N,123,Drama,8.6,7.0,[kuambil lagi hatiku],kuambil lagi hatiku,right_only
1842400,NaN,NaN,NaN,NaN,NaN,tt9916730,movie,6 gunn,6 gunn,0.0,2017.0,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn,right_only
1842401,NaN,NaN,NaN,NaN,NaN,tt9916730,movie,6 gunn,6 gunn,0.0,2017.0,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn,right_only
1842402,NaN,NaN,NaN,NaN,NaN,tt9916730,movie,6 gunn,6 gunn,0.0,2017.0,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn,right_only


In [102]:
no_year_both = no_year[no_year["_merge"]=="both"]
no_year_both = no_year_both.drop_duplicates(subset = ["index", "tconst"],keep = "first")
no_year_both["tconst"].nunique()

927

In [103]:
no_year_both['difference_in_years'] = no_year_both.apply(lambda x: abs(x['startYear'] - x['release_year']), axis=1)
no_year_both = no_year_both[no_year_both["difference_in_years"] <= 10]
no_year_both["tconst"].nunique()

465

In [104]:
no_year_both.head(50)

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge,difference_in_years
1,15.0,f1,Coffin Joe,the end of man,1970.0,tt0067099,movie,finis hominis,finis hominis,0.0,1971.0,\N,79,"Comedy,Drama,Mystery",5.7,373.0,"[end of man, the end of man, finis hominis, en...",the end of man,both,1.0
2,26.0,f2,The Crime Club,the last express,1938.0,tt0037776,movie,the hidden eye,the hidden eye,0.0,1945.0,\N,69,"Action,Crime,Mystery",6.2,397.0,"[the hidden eye, perfume do oriente, l'oeil ca...",the last express,both,7.0
73,185.0,f16,Young and Dangerous,those were the days,2000.0,tt0114146,movie,those were the days,le plus bel âge...,0.0,1995.0,\N,85,Drama,6.5,208.0,"[a mais bela idade, those were the days, najpi...",those were the days,both,5.0
75,185.0,f16,Young and Dangerous,those were the days,2000.0,tt0118165,movie,those were the days,wong gok dik tin hung 2: nam siu yee,0.0,1996.0,\N,89,"Action,Crime,Drama",5.1,17.0,"[those were the days, wong gok dik tin hung 2:...",those were the days,both,4.0
77,185.0,f16,Young and Dangerous,those were the days,2000.0,tt0186543,movie,si ge 32a he yi ge xiang jiao shao nian,si ge 32a he yi ge xiang jiao shao nian,0.0,1996.0,\N,101,Drama,6.6,32.0,"[si ge 32a he yi ge xiang jiao shao nian, thos...",those were the days,both,4.0
78,185.0,f16,Young and Dangerous,those were the days,2000.0,tt0285244,movie,those were the days,jing zhuang nan xiong nan di,0.0,1997.0,\N,103,Comedy,6.5,140.0,"[those were the days, 精裝難兄難弟, 精装难兄难弟, jing zhu...",those were the days,both,3.0
119,234.0,f20,Gamera,gamera the brave,2006.0,tt0467923,movie,gamera the brave,chiisaki yûsha-tachi: gamera,0.0,2005.0,\N,96,"Action,Adventure,Family",6.6,1298.0,"[гамера: маленькие герои, gamera: o genaios, 小...",gamera the brave,both,1.0
121,237.0,f21,The Hombre Lobo Series (a.k.a. The Waldemar Da...,los monstruos del terror,1969.0,tt0064687,movie,assignment terror,los monstruos del terror,0.0,1970.0,\N,87,"Horror,Sci-Fi",4.1,950.0,"[operación terror, dracula jagt frankenstein, ...",los monstruos del terror,both,1.0
123,238.0,f21,The Hombre Lobo Series (a.k.a. The Waldemar Da...,la furia del hombre lobo,1970.0,tt0065750,movie,fury of the wolfman,la furia del hombre lobo,0.0,1972.0,\N,86,Horror,3.7,875.0,"[varulven, ярость оборотня, fury of the wolfma...",la furia del hombre lobo,both,2.0
126,239.0,f21,The Hombre Lobo Series (a.k.a. The Waldemar Da...,la noche de walpurgis,1970.0,tt0066160,movie,the werewolf versus the vampire woman,la noche de walpurgis,0.0,1971.0,\N,86,Horror,5.3,1876.0,"[satan vs. the wolf man, la noche de walpurgis...",la noche de walpurgis,both,1.0


In [105]:
matched = pd.concat([all_merged_both, no_year_both])
matched = matched[matched["_merge"]=="both"]
matched.sort_values("numVotes", ascending = False).drop_duplicates(subset=["index", "franchise_id"], keep="first").sort_values(["index"])
matched

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge,difference_in_years
0,0.0,f0,The Aldrich Family,what a life,1939.0,tt0032123,movie,what a life,what a life,0.0,1939.0,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both,NaN
3,1.0,f0,The Aldrich Family,life with henry,1940.0,tt0033834,movie,life with henry,life with henry,0.0,1940.0,\N,80,"Comedy,Family,Music",6.0,49.0,"[life with henry, henry quería ir a alaska, he...",life with henry,both,NaN
6,2.0,f0,The Aldrich Family,henry aldrich for president,1941.0,tt0033708,movie,henry aldrich for president,henry aldrich for president,0.0,1941.0,\N,75,"Comedy,Family",6.6,146.0,"[henry aldrich para presidente, henry aldrich ...",henry aldrich for president,both,NaN
9,3.0,f0,The Aldrich Family,"henry aldrich, editor",1942.0,tt0034842,movie,"henry aldrich, editor","henry aldrich, editor",0.0,1942.0,\N,72,"Comedy,Drama,Family",6.4,150.0,"[henry periodista, henry aldrich, editor]","henry aldrich, editor",both,NaN
12,4.0,f0,The Aldrich Family,henry and dizzy,1942.0,tt0034844,movie,henry and dizzy,henry and dizzy,0.0,1942.0,\N,71,"Comedy,Family",7.2,58.0,[henry and dizzy],henry and dizzy,both,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4941,8069.0,f1342,Hopalong Cassidy (American-Western),sunset trail,1939.0,tt0023539,movie,sunset trail,sunset trail,0.0,1932.0,\N,62,"Action,Music,Romance",6.2,42.0,"[la fattoria maledetta, to monopati tou thanat...",sunset trail,both,7.0
4944,8069.0,f1342,Hopalong Cassidy (American-Western),sunset trail,1939.0,tt0030812,movie,sunset trail,sunset trail,0.0,1938.0,\N,69,Western,6.9,180.0,"[gentleman-cowboy, äventyret i silver city, ro...",sunset trail,both,1.0
4948,8093.0,f1342,Hopalong Cassidy (American-Western),lost canyon,1943.0,tt0034996,movie,lost canyon,lost canyon,0.0,1942.0,\N,61,Western,6.3,161.0,"[striden i dödsdalen, den sorte rytter, desfil...",lost canyon,both,1.0
4955,8103.0,f1342,Hopalong Cassidy (American-Western),fool's gold,1947.0,tt0038532,movie,fool's gold,fool's gold,0.0,1946.0,\N,63,"Drama,Western",6.1,140.0,"[överfallet på guldtransporten, twin buttes, r...",fool's gold,both,1.0


In [106]:
imdb_remainder = imdb_remainder[~imdb_remainder["tconst"].isin(all_merged_both["tconst"])]

In [107]:
def get_top_matches(row):
    return process.extract(row.movie_name, imdb_movie_list, limit = 3)

In [108]:
imdb_movie_list = imdb_remainder.all_names.to_list()

In [109]:
#wiki_remainder['best_match'] = wiki_remainder.apply(lambda row: get_top_matches(row), axis = 1)

In [110]:
matched = matched[["index", "franchise_id", "franchise_name", "movie_name", "tconst", "primaryTitle", "isAdult", "release_year", "runtimeMinutes", "genres", "averageRating", "numVotes"]]

In [111]:
matched.columns = matched.columns.str.replace('release_year', 'startYear')

In [112]:
matched["startYear"] = matched["startYear"].apply(lambda x : int(x))

<ipython-input-112-d53acfba8579>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched["startYear"] = matched["startYear"].apply(lambda x : int(x))


In [113]:
imdb_final = pd.merge(matched, imdb_movies, on = "tconst", how = "outer")
imdb_final

,index,franchise_id,franchise_name,movie_name,tconst,primaryTitle_x,isAdult_x,startYear_x,runtimeMinutes_x,genres_x,...,primaryTitle_y,originalTitle,isAdult_y,startYear_y,endYear,runtimeMinutes_y,genres_y,averageRating_y,numVotes_y,akas
0,0.0,f0,The Aldrich Family,what a life,tt0032123,what a life,0.0,1939.0,75,"Comedy,Drama",...,what a life,what a life,0,1939.0,\N,75,"Comedy,Drama",6.9,93.0,"['what a life', 'a vida começa aos 14']"
1,1.0,f0,The Aldrich Family,life with henry,tt0033834,life with henry,0.0,1940.0,80,"Comedy,Family,Music",...,life with henry,life with henry,0,1940.0,\N,80,"Comedy,Family,Music",6.0,49.0,"['life with henry', 'henry quería ir a alaska'..."
2,2.0,f0,The Aldrich Family,henry aldrich for president,tt0033708,henry aldrich for president,0.0,1941.0,75,"Comedy,Family",...,henry aldrich for president,henry aldrich for president,0,1941.0,\N,75,"Comedy,Family",6.6,146.0,"['henry aldrich para presidente', 'henry aldri..."
3,3.0,f0,The Aldrich Family,"henry aldrich, editor",tt0034842,"henry aldrich, editor",0.0,1942.0,72,"Comedy,Drama,Family",...,"henry aldrich, editor","henry aldrich, editor",0,1942.0,\N,72,"Comedy,Drama,Family",6.4,150.0,"['henry periodista', 'henry aldrich, editor']"
4,4.0,f0,The Aldrich Family,henry and dizzy,tt0034844,henry and dizzy,0.0,1942.0,71,"Comedy,Family",...,henry and dizzy,henry and dizzy,0,1942.0,\N,71,"Comedy,Family",7.2,58.0,['henry and dizzy']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290383,NaN,NaN,NaN,NaN,tt9916270,NaN,NaN,NaN,NaN,NaN,...,il talento del calabrone,il talento del calabrone,0,2020.0,\N,84,Thriller,5.8,1414.0,"['il talento del calabrone', 'the talent of th..."
290384,NaN,NaN,NaN,NaN,tt9916362,NaN,NaN,NaN,NaN,NaN,...,coven,akelarre,0,2020.0,\N,92,"Drama,History",6.4,5293.0,"['boszorkánygyülekezet', 'coven', ""les sorcièr..."
290385,NaN,NaN,NaN,NaN,tt9916428,NaN,NaN,NaN,NaN,NaN,...,the secret of china,hong xing zhao yao zhong guo,0,2019.0,\N,\N,"Adventure,History,War",3.8,14.0,"['the secret of china', 'hong xing zhao yao zh..."
290386,NaN,NaN,NaN,NaN,tt9916538,NaN,NaN,NaN,NaN,NaN,...,kuambil lagi hatiku,kuambil lagi hatiku,0,2019.0,\N,123,Drama,8.6,7.0,['kuambil lagi hatiku']


In [114]:
imdb_final = imdb_final[["index", "franchise_id", "franchise_name", "movie_name", "tconst", "primaryTitle_y", "isAdult_y", "startYear_y", "runtimeMinutes_y", "genres_y", "averageRating_y", "numVotes_y"]]
imdb_final

,index,franchise_id,franchise_name,movie_name,tconst,primaryTitle_y,isAdult_y,startYear_y,runtimeMinutes_y,genres_y,averageRating_y,numVotes_y
0,0.0,f0,The Aldrich Family,what a life,tt0032123,what a life,0,1939.0,75,"Comedy,Drama",6.9,93.0
1,1.0,f0,The Aldrich Family,life with henry,tt0033834,life with henry,0,1940.0,80,"Comedy,Family,Music",6.0,49.0
2,2.0,f0,The Aldrich Family,henry aldrich for president,tt0033708,henry aldrich for president,0,1941.0,75,"Comedy,Family",6.6,146.0
3,3.0,f0,The Aldrich Family,"henry aldrich, editor",tt0034842,"henry aldrich, editor",0,1942.0,72,"Comedy,Drama,Family",6.4,150.0
4,4.0,f0,The Aldrich Family,henry and dizzy,tt0034844,henry and dizzy,0,1942.0,71,"Comedy,Family",7.2,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...
290383,NaN,NaN,NaN,NaN,tt9916270,il talento del calabrone,0,2020.0,84,Thriller,5.8,1414.0
290384,NaN,NaN,NaN,NaN,tt9916362,coven,0,2020.0,92,"Drama,History",6.4,5293.0
290385,NaN,NaN,NaN,NaN,tt9916428,the secret of china,0,2019.0,\N,"Adventure,History,War",3.8,14.0
290386,NaN,NaN,NaN,NaN,tt9916538,kuambil lagi hatiku,0,2019.0,123,Drama,8.6,7.0


In [115]:
imdb_final.columns = imdb_final.columns.str.replace('primaryTitle_y', 'primaryTitle')
imdb_final.columns = imdb_final.columns.str.replace('isAdult_y', 'isAdult')
imdb_final.columns = imdb_final.columns.str.replace('startYear_y', 'startYear')
imdb_final.columns = imdb_final.columns.str.replace('runtimeMinutes_y', 'runtimeMinutes')
imdb_final.columns = imdb_final.columns.str.replace('genres_y', 'genres')
imdb_final.columns = imdb_final.columns.str.replace('averageRating_y', 'averageRating')
imdb_final.columns = imdb_final.columns.str.replace('numVotes_y', 'numVotes')

In [116]:
imdb_final[imdb_final["primaryTitle"] == "iron man"]

,index,franchise_id,franchise_name,movie_name,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
4389,7429.0,f1329,Marvel Cinematic Universe (various),iron man,tt0371746,iron man,0,2008.0,126,"Action,Adventure,Sci-Fi",7.9,1076204.0
9971,NaN,NaN,NaN,NaN,tt0022002,iron man,0,1931.0,73,"Drama,Romance",5.8,322.0
22941,NaN,NaN,NaN,NaN,tt0043678,iron man,0,1951.0,82,"Drama,Film-Noir,Sport",6.3,247.0
234065,NaN,NaN,NaN,NaN,tt3375556,iron man,0,2013.0,84,"Comedy,Drama,Family",5.0,30.0


In [117]:
wiki_remainder["franchise_name"].unique()

array(['Coffin Joe', 'The Crime Club', 'Fast & Furious',
       'Gingerdead Man vs. Evil Bong', 'Lash LaRue', 'Love Comes Softly',
       'Madame Aema', 'Mickey Mouse', 'Mystery Woman', 'Naruto the Movie',
       'Wizarding World', 'Yogi Bear', 'Hanna-Barbera Superstars 10',
       'Young and Dangerous', 'Aurora Teagarden Mystery',
       'Bomba, the Jungle Boy', 'The Flintstones', 'The Jetsons',
       'Gamera',
       'The Hombre Lobo Series (a.k.a. The Waldemar Daninsky Series)',
       'L.E.T.H.A.L. Ladies (a.k.a. Triple-B, Bullets, Bombs and Babes)',
       'Michael Shayne', 'The Muppets',
       'Nemuri Kyoshirō (Matsukata Hiroki series)',
       'Signed, Sealed, Delivered', 'American Girl', 'Bratz', 'Mothra',
       'Pekka and Pätkä', 'Schulmädchen-Report', 'Wangan Middonaito',
       'Air Bud', 'American Film Theatre', 'Darna', 'FIFA World Cup',
       'The Land Before Time', 'Olsen-banden (Danish series)',
       'Olsenbanden (Norwegian series)', 'One Piece',
       'Sherlock 

In [118]:
test = imdb_copy[imdb_copy["primaryTitle"].str.contains("harry potter and the deathly", na=False, flags=re.IGNORECASE, regex=True)]
test

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names
144170,tt0926084,movie,harry potter and the deathly hallows: part 1,harry potter and the deathly hallows: part 1,0,2010.0,\N,146,"Adventure,Family,Fantasy",7.7,564206.0,"[harry potter and the deathly hallows: part 1,...",harry potter and the deathly hallows: part 1
144170,tt0926084,movie,harry potter and the deathly hallows: part 1,harry potter and the deathly hallows: part 1,0,2010.0,\N,146,"Adventure,Family,Fantasy",7.7,564206.0,"[harry potter and the deathly hallows: part 1,...",harry potter and the deathly hallows: part 1
144170,tt0926084,movie,harry potter and the deathly hallows: part 1,harry potter and the deathly hallows: part 1,0,2010.0,\N,146,"Adventure,Family,Fantasy",7.7,564206.0,"[harry potter and the deathly hallows: part 1,...",harry potter and the deathly hallows: part 1
144170,tt0926084,movie,harry potter and the deathly hallows: part 1,harry potter and the deathly hallows: part 1,0,2010.0,\N,146,"Adventure,Family,Fantasy",7.7,564206.0,"[harry potter and the deathly hallows: part 1,...",haris poteris ir mirties relikvijos. 1 dalis 3d
144170,tt0926084,movie,harry potter and the deathly hallows: part 1,harry potter and the deathly hallows: part 1,0,2010.0,\N,146,"Adventure,Family,Fantasy",7.7,564206.0,"[harry potter and the deathly hallows: part 1,...","harry potter ja kuoleman varjelukset, osa 1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161732,tt1201607,movie,harry potter and the deathly hallows: part 2,harry potter and the deathly hallows: part 2,0,2011.0,\N,130,"Adventure,Family,Fantasy",8.1,894796.0,"[harry potter and the deathly hallows: part 2,...",harry potter et les reliques de la mort : part...
161732,tt1201607,movie,harry potter and the deathly hallows: part 2,harry potter and the deathly hallows: part 2,0,2011.0,\N,130,"Adventure,Family,Fantasy",8.1,894796.0,"[harry potter and the deathly hallows: part 2,...",harry potter và bảo bối tử thần: phần 2
161732,tt1201607,movie,harry potter and the deathly hallows: part 2,harry potter and the deathly hallows: part 2,0,2011.0,\N,130,"Adventure,Family,Fantasy",8.1,894796.0,"[harry potter and the deathly hallows: part 2,...",哈利·波特与死亡圣器下
161732,tt1201607,movie,harry potter and the deathly hallows: part 2,harry potter and the deathly hallows: part 2,0,2011.0,\N,130,"Adventure,Family,Fantasy",8.1,894796.0,"[harry potter and the deathly hallows: part 2,...","harry potter ja kuoleman varjelukset, osa 2"


In [119]:
imdb_final.to_csv("movies_with_franchises.csv")

# Test Code

In [52]:
"Wizarding World" 

'Wizarding World'

In [53]:
wiki_remainder[wiki_remainder["franchise_name"] == "DC Extended Universe"]

,index,franchise_id,franchise_name,movie_name,release_year
1389,629.0,f50,DC Extended Universe,the flash,2023.0
1390,630.0,f50,DC Extended Universe,blue beetle,2023.0
1391,631.0,f50,DC Extended Universe,aquaman and the lost kingdom,2023.0


In [67]:
test = imdb_remainder.sort_values("numVotes", ascending = False).drop_duplicates(subset=["tconst"], keep="first").sort_values(["tconst"])
test[test["numVotes"] >292]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names
16340,tt0000147,movie,the corbett-fitzsimmons fight,the corbett-fitzsimmons fight,0.0,1897.0,\N,100,"Documentary,News,Sport",5.3,469.0,"[the corbett-fitzsimmons fight, бой корбетта и...",бой корбетта и фитцсиммонса
16350,tt0000574,movie,the story of the kelly gang,the story of the kelly gang,0.0,1906.0,\N,70,"Action,Adventure,Biography",6.0,823.0,"[the story of the kelly gang, kelly bandájának...",die geschichte der kelly bande
16612,tt0002101,movie,cleopatra,cleopatra,0.0,1912.0,\N,100,"Drama,History",5.1,553.0,"[cleopatra, κλεοπάτρα, клеопатра, cleopatra: t...",клеопатра
16624,tt0002130,movie,dante's inferno,l'inferno,0.0,1911.0,\N,71,"Adventure,Drama,Fantasy",7.0,3117.0,"[helvetti, a pokol, l'inferno, インフェルノ, dante's...",l'enfer
16658,tt0002199,movie,from the manger to the cross,from the manger to the cross,0.0,1912.0,\N,60,"Biography,Drama",5.8,615.0,"[da manjedoura à cruz, från krubban till korse...",from the manger to the cross
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920204,tt9908390,movie,le lion,le lion,0.0,2020.0,\N,95,Comedy,5.4,1284.0,"[špijun lavljeg srca, agente super secreto, le...",špijun lavljeg srca
1920263,tt9911196,movie,the marriage escape,de beentjes van sint-hildegard,0.0,2020.0,\N,103,"Comedy,Drama",7.5,2885.0,"[the marriage escape, 求分大作戰, demasiado amor, s...",the marriage escape
1920279,tt9911774,movie,padmavyuhathile abhimanyu,padmavyuhathile abhimanyu,0.0,2019.0,\N,130,Drama,8.1,379.0,"[padmavyuhathile abhimanyu, pathmavyuhathile a...",padmavyoohathile abhimanyu
1920365,tt9916270,movie,il talento del calabrone,il talento del calabrone,0.0,2020.0,\N,84,Thriller,5.8,1414.0,"[il talento del calabrone, the talent of the h...",il calabrone


In [65]:
test["numVotes"].describe()

count    2.855470e+05
mean     2.798095e+03
std      2.804832e+04
min      5.000000e+00
25%      1.800000e+01
50%      5.800000e+01
75%      2.920000e+02
max      2.727553e+06
Name: numVotes, dtype: float64

In [68]:
matched[matched["numVotes"] >292]

,index,franchise_id,franchise_name,movie_name,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
34,12.0,f1,Coffin Joe,this night i'll possess your corpse,tt0060380,esta noite encarnarei no teu cadáver,0.0,1967,108,Horror,7.0,1952.0
35,13.0,f1,Coffin Joe,the strange world of coffin joe,tt0062943,the strange world of coffin joe,0.0,1968,80,Horror,6.3,653.0
37,14.0,f1,Coffin Joe,awakening of the beast,tt0062690,awakening of the beast,0.0,1970,93,"Drama,Horror",5.8,1208.0
44,19.0,f1,Coffin Joe,hallucinations of a deranged mind,tt0077420,hallucinations of a deranged mind,0.0,1978,86,Horror,5.2,379.0
47,21.0,f1,Coffin Joe,embodiment of evil,tt0923683,embodiment of evil,0.0,2008,94,Horror,5.9,1712.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4709,7726.0,f1335,The Bowery Boys,dig that uranium,tt0049143,dig that uranium,0.0,1956,61,"Comedy,Family",6.1,310.0
4722,7748.0,f1336,Edgar Wallace Mysteries,backfire,tt0058785,backfire,0.0,1961,94,"Adventure,Comedy,Crime",6.1,320.0
4882,7924.0,f1339,The Three Mesquiteers,westward ho,tt0027200,westward ho,0.0,1942,61,Western,5.6,599.0
4906,7951.0,f1340,Santo,santo en el tesoro de drácula,tt0208504,santo in the treasure of dracula,0.0,1968,81,"Action,Adventure,Fantasy",5.0,453.0


In [59]:
#imdb_remainder["numVotes"].describe()
imdb_remainder[imdb_remainder["numVotes"] >100]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names
16333,tt0000009,movie,miss jerry,miss jerry,0.0,1894.0,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]",miss jerry
16334,tt0000009,movie,miss jerry,miss jerry,0.0,1894.0,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]",miss jerry
16335,tt0000009,movie,miss jerry,miss jerry,0.0,1894.0,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]",miss jerry
16336,tt0000009,movie,miss jerry,miss jerry,0.0,1894.0,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]",fräulein jerry
16337,tt0000147,movie,the corbett-fitzsimmons fight,the corbett-fitzsimmons fight,0.0,1897.0,\N,100,"Documentary,News,Sport",5.3,469.0,"[the corbett-fitzsimmons fight, бой корбетта и...",the corbett-fitzsimmons fight
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920376,tt9916362,movie,coven,akelarre,0.0,2020.0,\N,92,"Drama,History",6.4,5293.0,"[boszorkánygyülekezet, coven, les sorcières d'...",kurtuluş ayini
1920377,tt9916362,movie,coven,akelarre,0.0,2020.0,\N,92,"Drama,History",6.4,5293.0,"[boszorkánygyülekezet, coven, les sorcières d'...",tanz der unschuldigen
1920378,tt9916362,movie,coven,akelarre,0.0,2020.0,\N,92,"Drama,History",6.4,5293.0,"[boszorkánygyülekezet, coven, les sorcières d'...",sabat sióstr
1920379,tt9916362,movie,coven,akelarre,0.0,2020.0,\N,92,"Drama,History",6.4,5293.0,"[boszorkánygyülekezet, coven, les sorcières d'...",akelarre: o ritual da irmandade


In [58]:
imdb_remainder

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names
16333,tt0000009,movie,miss jerry,miss jerry,0.0,1894.0,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]",miss jerry
16334,tt0000009,movie,miss jerry,miss jerry,0.0,1894.0,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]",miss jerry
16335,tt0000009,movie,miss jerry,miss jerry,0.0,1894.0,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]",miss jerry
16336,tt0000009,movie,miss jerry,miss jerry,0.0,1894.0,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]",fräulein jerry
16337,tt0000147,movie,the corbett-fitzsimmons fight,the corbett-fitzsimmons fight,0.0,1897.0,\N,100,"Documentary,News,Sport",5.3,469.0,"[the corbett-fitzsimmons fight, бой корбетта и...",the corbett-fitzsimmons fight
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920387,tt9916538,movie,kuambil lagi hatiku,kuambil lagi hatiku,0.0,2019.0,\N,123,Drama,8.6,7.0,[kuambil lagi hatiku],kuambil lagi hatiku
1920388,tt9916730,movie,6 gunn,6 gunn,0.0,2017.0,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn
1920389,tt9916730,movie,6 gunn,6 gunn,0.0,2017.0,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn
1920390,tt9916730,movie,6 gunn,6 gunn,0.0,2017.0,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn


In [71]:
imdb_final[imdb_final["franchise_id"].isnull]

TypeError: isnull() takes 1 positional argument but 2 were given

In [77]:
test = pd.isnull(imdb_final["franchise_id"])
imdb_final[test]["numVotes"].describe()

count    2.850820e+05
mean     2.792056e+03
std      2.805198e+04
min      5.000000e+00
25%      1.800000e+01
50%      5.800000e+01
75%      2.910000e+02
max      2.727553e+06
Name: numVotes, dtype: float64

In [89]:
matched

,index,franchise_id,franchise_name,movie_name,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,0.0,f0,The Aldrich Family,what a life,tt0032123,what a life,0.0,1939,75,"Comedy,Drama",6.9,93.0
3,1.0,f0,The Aldrich Family,life with henry,tt0033834,life with henry,0.0,1940,80,"Comedy,Family,Music",6.0,49.0
6,2.0,f0,The Aldrich Family,henry aldrich for president,tt0033708,henry aldrich for president,0.0,1941,75,"Comedy,Family",6.6,146.0
9,3.0,f0,The Aldrich Family,"henry aldrich, editor",tt0034842,"henry aldrich, editor",0.0,1942,72,"Comedy,Drama,Family",6.4,150.0
12,4.0,f0,The Aldrich Family,henry and dizzy,tt0034844,henry and dizzy,0.0,1942,71,"Comedy,Family",7.2,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4941,8069.0,f1342,Hopalong Cassidy (American-Western),sunset trail,tt0023539,sunset trail,0.0,1939,62,"Action,Music,Romance",6.2,42.0
4944,8069.0,f1342,Hopalong Cassidy (American-Western),sunset trail,tt0030812,sunset trail,0.0,1939,69,Western,6.9,180.0
4948,8093.0,f1342,Hopalong Cassidy (American-Western),lost canyon,tt0034996,lost canyon,0.0,1943,61,Western,6.3,161.0
4955,8103.0,f1342,Hopalong Cassidy (American-Western),fool's gold,tt0038532,fool's gold,0.0,1947,63,"Drama,Western",6.1,140.0


In [91]:
matched[matched["franchise_name"] == "Marvel Cinematic Universe (various)"]

,index,franchise_id,franchise_name,movie_name,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
1282,7444.0,f1329,Marvel Cinematic Universe (various),spider-man: homecoming,tt2250912,spider-man: homecoming,0.0,2017,133,"Action,Adventure,Sci-Fi",7.4,674123.0
1295,7451.0,f1329,Marvel Cinematic Universe (various),spider-man: far from home,tt6320628,spider-man: far from home,0.0,2019,129,"Action,Adventure,Comedy",7.4,510074.0
1305,7455.0,f1329,Marvel Cinematic Universe (various),spider-man: no way home,tt10872600,spider-man: no way home,0.0,2021,148,"Action,Adventure,Fantasy",8.2,785467.0
14787,7429.0,f1329,Marvel Cinematic Universe (various),iron man,tt0371746,iron man,0.0,2008,126,"Action,Adventure,Sci-Fi",7.9,1076204.0
14790,7430.0,f1329,Marvel Cinematic Universe (various),the incredible hulk,tt0800080,the incredible hulk,0.0,2008,112,"Action,Adventure,Sci-Fi",6.6,502385.0
14793,7431.0,f1329,Marvel Cinematic Universe (various),iron man 2,tt1228705,iron man 2,0.0,2010,124,"Action,Sci-Fi",6.9,831954.0
14796,7432.0,f1329,Marvel Cinematic Universe (various),thor,tt0800369,thor,0.0,2011,115,"Action,Fantasy",7.0,863384.0
14799,7433.0,f1329,Marvel Cinematic Universe (various),captain america: the first avenger,tt0458339,captain america: the first avenger,0.0,2011,124,"Action,Adventure,Sci-Fi",6.9,859649.0
14802,7434.0,f1329,Marvel Cinematic Universe (various),the avengers,tt0848228,the avengers,0.0,2012,143,"Action,Sci-Fi",8.0,1409220.0
14805,7435.0,f1329,Marvel Cinematic Universe (various),iron man 3,tt1300854,iron man 3,0.0,2013,130,"Action,Adventure,Sci-Fi",7.1,864689.0


In [78]:
test = pd.notnull(imdb_final["franchise_id"])
imdb_final[test]["numVotes"].describe()

count    5.306000e+03
mean     5.207886e+04
std      1.690172e+05
min      5.000000e+00
25%      1.520000e+02
50%      1.005500e+03
75%      1.177825e+04
max      2.700528e+06
Name: numVotes, dtype: float64

# Unused code

In [ ]:
list1 = list(matched["index"])
found_mask = []
for i in range(0, len(list(wiki_movies["index"]))):
    if i in list1:
        found_mask.append(True)
    else:
        found_mask.append(False)
    
#print(list1)
print(wiki_movies[found_mask])

In [ ]:
# apply this mask function at each step of the matching process and see how many found rows there are

In [ ]:
3450 + 4749

In [ ]:
matched["index"].value_counts()

In [ ]:
both_test = matched[matched["_merge"]=="both"].sort_values(by = "index")

In [ ]:
dict(both_test["franchise_name"].value_counts())

In [ ]:
# look into moving all names into one list, then searching with date
# With remainder, search without date and see what happens

In [ ]:
# losing some values in both data frames for unknown reason when using merge.
# starts with 6405 rows in wiki and 290239 rows in imdb
# 3307 
#              wiki  |  imdb 
# start     |  6405  |  290239
# found     |  3088  |    3088
# remaining |  3307  |  287199
# total     |  6395  |  290287
# variance  |   -10  |     +48

### Checks for movies in dataframes

In [ ]:
test = both_test[both_test["franchise_name"].str.contains("Edgar Wallace Mysteries", na=False, flags=re.IGNORECASE, regex=True)]
test.head(50)

In [ ]:
test = wiki_movies[wiki_movies["franchise_name"].str.contains("Edgar Wallace Mysteries", na=False, flags=re.IGNORECASE, regex=True)]
test

In [ ]:
test = imdb_movies[imdb_movies["primaryTitle"].str.contains("star wars", na=False, flags=re.IGNORECASE, regex=True)]
test

In [ ]:
test = matched[matched["index"] == 5191.0]
test

In [ ]:
test = imdb_copy[imdb_copy["tconst"].str.contains("tt5433140", na=False, flags=re.IGNORECASE, regex=True)]
test.head()


In [ ]:
imdb_movies['titles'] = imdb_movies.apply(', '.join(imdb_movies.columns["primaryTitle", "originalTitle", "akas"]), axis = 1)
imdb_movies

In [ ]:
def fuzzy_match(wiki, imdb, threshold):
    # iterate through each row in wiki
    # 
    # 
    # 
    # 
    # 
    # 
    # 

In [ ]:
fuzzy_match(wiki_movies, imdb_movies, 95)

In [ ]:
wiki_movies[wiki_movies["franchise_id"] == "f1312"]

In [ ]:
imdb_movies[imdb_movies["tconst"] == "tt3262112"]